In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 
import metpy

# 2021.11.09 unfinished
尝试单独将UV插值到等高面上，然后进行速度方差谱的计算绘制，相加得到动能谱

分开进行插值部分

## 数据读入

In [ ]:
winds = {}
hgts  = {}

# ERA5I
dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/u_98-17_daily/"
ds_u    = xr.open_dataset(dir_in + "merge_u_daily.nc") # 自动合并 Time 坐标
winds['era5i'] = {}
origin_time = ds_u.time
jja_time    = origin_time.isel(time = origin_time.dt.month.isin([6,7,8]))
winds['era5i']['uwnd'] = ds_u['uwnd'].sel(time = jja_time)

dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/v_98-17_daily/"
ds_v    = xr.open_dataset(dir_in + "merge_v_daily.nc") # 自动合并 Time 坐标
winds['era5i']['vwnd'] = ds_v['vwnd'].sel(time = jja_time)

dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/hgt/"
ds_hgt    = xr.open_dataset(dir_in + "merge_hgt_daily.nc") # 自动合并 Time 坐标
hgts['era5i']      = ds_hgt['hgt'].sel(time = jja_time)

# VR & RCM uReconstructZonal uReconstructMeridional
model_type = "VR"
dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/ke_daily_vi/"
ds_ke    = xr.open_mfdataset(dir_in + "*_"+model_type+"_ke_daily_vi.nc") # 自动合并 Time 坐标
origin_time = ds_ke.Time
jja_time    = origin_time.isel(Time = origin_time.dt.month.isin([6,7,8]))
winds['vr'] = {}
winds['vr']['uwnd'] = ds_ke['uReconstructZonal'].sel(Time = jja_time)
winds['vr']['vwnd'] = ds_ke['uReconstructMeridional'].sel(Time = jja_time)

dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/hgt_daily_vi/"
ds_hgt = xr.open_mfdataset(dir_in + "*" + model_type + "_hgt_daily_vi.nc")
hgts['vr']      = ds_hgt['zgrid'].sel(Time = jja_time)

model_type = "RCM"
dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/ke_daily_vi/"
ds_ke    = xr.open_mfdataset(dir_in + "*_"+model_type+"_ke_daily_vi.nc") # 自动合并 Time 坐标
winds['rcm'] = {}
winds['rcm']['uwnd'] = ds_ke['uReconstructZonal'].sel(Time = jja_time)
winds['rcm']['vwnd'] = ds_ke['uReconstructMeridional'].sel(Time = jja_time)
dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/hgt_daily_vi/"
ds_hgt = xr.open_mfdataset(dir_in + "*" + model_type + "_hgt_daily_vi.nc")
hgts['rcm']      = ds_hgt['zgrid'].sel(Time = jja_time)

## 垂直插值 & 写出

### ERA5I

In [ ]:
from metpy.units import units
from metpy.interpolate import * # log_interpolate_1d
hgt_test = hgts['era5i'][:,:,:,:]   #['hgt']  #[:,:,:,:]
uwnd_test  = winds['era5i']['uwnd'][:,:,:,:]  #[0,:,:,:]
vwnd_test  = winds['era5i']['vwnd'][:,:,:,:]  #[0,:,:,:]
vi_height = xr.DataArray(np.array([8500,9500,10500,11500]), name = "height", dims = 'height')
uwnd_height = interpolate_1d(vi_height.values, hgt_test.values, uwnd_test.values, axis= 1) # 1 for ERA5I  3 for MPAS
vwnd_height = interpolate_1d(vi_height.values, hgt_test.values, vwnd_test.values, axis= 1) # 1 for ERA5I  3 for MPAS

In [ ]:
uwnd_vi_rmnan = np.where(uwnd_height>1e10, -1e30, uwnd_height)
vwnd_vi_rmnan = np.where(vwnd_height>1e10, -1e30, vwnd_height)
coords_lon    = uwnd_test.lon
coords_lat    = uwnd_test.lat
coords_time   = uwnd_test.time
uwnd_xr = xr.DataArray(uwnd_vi_rmnan, name = "uwnd", coords = [coords_time, vi_height, coords_lat ,coords_lon],attrs={"caculation":"8.5km-10.5km average Ke trapz integration"})
vwnd_xr = xr.DataArray(vwnd_vi_rmnan, name = "vwnd", coords = [coords_time, vi_height, coords_lat ,coords_lon],attrs={"caculation":"8.5km-10.5km average Ke trapz integration"})
ds_write = uwnd_xr.to_dataset()
ds_write[vwnd_xr.name] = vwnd_xr
ds_write
# ds_write['vwnd'][10,0,:,:].plot()

In [ ]:
path_out = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/allwinds_daily/"
encoding_dict = {"uwnd":{"_FillValue":-1e30},"vwnd":{"_FillValue":-1e30}}
ds_write.to_netcdf(path_out + "allwinds_vi_8.5-11.5km.nc",'w',encoding = encoding_dict)

### vr & rcm

In [ ]:
from metpy.units import units
from metpy.interpolate import * # log_interpolate_1d
hgt_test = hgts['rcm'][:,:,:,:]   #['hgt']  #[:,:,:,:]
uwnd_test  = winds['rcm']['uwnd'][:,:,:,:]  #[0,:,:,:]
vwnd_test  = winds['rcm']['vwnd'][:,:,:,:]  #[0,:,:,:]
vi_height = xr.DataArray(np.array([8500,9500,10500,11500]), name = "height", dims = 'height')
uwnd_height = interpolate_1d(vi_height.values, hgt_test.values, uwnd_test.values, axis= 3) # 1 for ERA5I  3 for MPAS
vwnd_height = interpolate_1d(vi_height.values, hgt_test.values, vwnd_test.values, axis= 3) # 1 for ERA5I  3 for MPAS

In [ ]:
uwnd_vi_rmnan = np.where(uwnd_height>1e10, -1e30, uwnd_height)
vwnd_vi_rmnan = np.where(vwnd_height>1e10, -1e30, vwnd_height)

uwnd_vi_rmnan = np.moveaxis(uwnd_height,3,1)
vwnd_vi_rmnan = np.moveaxis(vwnd_height,3,1)

coords_lon  = uwnd_test.longitude
coords_lat  = uwnd_test.latitude
coords_time = uwnd_test.Time
uwnd_xr = xr.DataArray(uwnd_vi_rmnan, name = "uwnd", coords = [coords_time, vi_height, coords_lat ,coords_lon],attrs={"caculation":"8.5km-10.5km average Ke trapz integration"})
vwnd_xr = xr.DataArray(vwnd_vi_rmnan, name = "vwnd", coords = [coords_time, vi_height, coords_lat ,coords_lon],attrs={"caculation":"8.5km-10.5km average Ke trapz integration"})
ds_write = uwnd_xr.to_dataset()
ds_write[vwnd_xr.name] = vwnd_xr
ds_write

In [ ]:
path_out = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge/allwinds_daily/"
encoding_dict = {"uwnd":{"_FillValue":-1e30},"vwnd":{"_FillValue":-1e30}}
ds_write.to_netcdf(path_out + "allwinds_vi_8.5-11.5km.nc",'w',encoding = encoding_dict)